<a href="https://colab.research.google.com/github/girisoura/automatic-octo-robot/blob/main/Keras_hyper_Tuner_Pima__Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('diabetes.csv')

In [3]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler=StandardScaler()
X=scaler.fit_transform(x)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [15]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),batch_size=32)

Epoch 1/100
20/20 [==============================] - 1s 13ms/step - loss: 0.5849 - accuracy: 0.7264 - val_loss: 0.5698 - val_accuracy: 0.7208
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5421 - accuracy: 0.7573 - val_loss: 0.5454 - val_accuracy: 0.7403
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5171 - accuracy: 0.7606 - val_loss: 0.5322 - val_accuracy: 0.7597
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4998 - accuracy: 0.7557 - val_loss: 0.5250 - val_accuracy: 0.7597
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4898 - accuracy: 0.7573 - val_loss: 0.5200 - val_accuracy: 0.7597
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4814 - accuracy: 0.7557 - val_loss: 0.5162 - val_accuracy: 0.7662
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4752 - accuracy: 0.7638 - val_loss: 0.5157 - val_accuracy: 0.7662
Epoch 8/100


In [17]:
#1.how to select appropriete optimizer

In [20]:
import kerastuner as kt

<ipython-input-20-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [23]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [24]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [27]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [28]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [29]:
model=tuner.get_best_models(num_models=1)[0]

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4911 - accuracy: 0.7476 - val_loss: 0.5205 - val_accuracy: 0.7468
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4811 - accuracy: 0.7524 - val_loss: 0.5158 - val_accuracy: 0.7727
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4738 - accuracy: 0.7573 - val_loss: 0.5100 - val_accuracy: 0.7727
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4680 - accuracy: 0.7557 - val_loss: 0.5074 - val_accuracy: 0.7662
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4634 - accuracy: 0.7622 - val_loss: 0.5053 - val_accuracy: 0.7662
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4589 - accuracy: 0.7687 - val_loss: 0.5044 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4549 - accuracy: 0.7638 - val_loss: 0.5052 - val_accuracy: 0.7727
Epoch 8/100


In [40]:
def build_model(hp):
  model=Sequential()
  units=hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [41]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='giri')
tuner

In [47]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))
